In [1]:
import pandas as pd

col_names = ['tweethash','sentiment','text']
data_path = 'SemEval2017-sentiment_english.csv'

tweet_data = pd.read_csv(data_path, header=None, names=col_names, encoding="ISO-8859-1").sample(frac=1) # .sample(frac=1) shuffles the data
tweet_data = tweet_data[['sentiment', 'text']] # Disregard other columns
print(tweet_data.head())

               sentiment                                               text
9249   __label__positive  can we get him to beat out the wanted atleast ...
4673    __label__neutral  My heart felt condolences go out to the Giffor...
6914    __label__neutral  =@bgregory857 You may not know this but Americ...
10382  __label__positive  I found out i was gay in the 4th grade...well ...
15004   __label__neutral  WWE Monday Night RAW results - http://t.co/Tnn...


In [2]:
import re

allowed_chars = ' AaBbCcDdEeFfGgHhIiJjKkLlMmNnOoPpQqRrSsTtUuVvWwXxYyZz0123456789~`!@#$%^&*()-=_+[]{}|;:",./<>?'
punct = '!?,.@#'
maxlen = 280

def preprocess(text):
    return ''.join([' ' + char + ' ' if char in punct else char for char in [char for char in re.sub(r'http\S+', 'http', text, flags=re.MULTILINE) if char in allowed_chars]])[:maxlen]


In [3]:
tweet_data['text'] = tweet_data['text'].apply(preprocess)

In [4]:
#tweet_data['sentiment'] = '__label__' + tweet_data['sentiment'].astype(str)

In [5]:
import os

# Create directory for saving data if it does not already exist
data_dir = './processed-data'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

# Save a percentage of the data (you could also only load a fraction of the data instead)
amount = 0.125

tweet_data.iloc[0:int(len(tweet_data)*0.8*amount)].to_csv(data_dir + '/train.csv', sep='\t', index=False, header=False)
tweet_data.iloc[int(len(tweet_data)*0.8*amount):int(len(tweet_data)*0.9*amount)].to_csv(data_dir + '/test.csv', sep='\t', index=False, header=False)
tweet_data.iloc[int(len(tweet_data)*0.9*amount):int(len(tweet_data)*1.0*amount)].to_csv(data_dir + '/dev.csv', sep='\t', index=False, header=False)


In [6]:
from flair.data_fetcher import NLPTaskDataFetcher
from pathlib import Path

#corpus = NLPTaskDataFetcher.load_classification_corpus(Path(data_dir), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')


from flair.data import Corpus
from flair.datasets import ClassificationCorpus

# this is the folder in which train, test and dev files reside


# load corpus containing training, test and dev data
corpus: Corpus = ClassificationCorpus(Path(data_dir))

2020-03-09 23:17:32,837 Reading data from processed-data
2020-03-09 23:17:32,838 Train: processed-data/train.csv
2020-03-09 23:17:32,838 Dev: processed-data/dev.csv
2020-03-09 23:17:32,839 Test: processed-data/test.csv


In [7]:
label_dict = corpus.make_label_dictionary()

2020-03-09 23:17:33,946 Computing label dictionary. Progress:


100%|██████████| 160000/160000 [00:30<00:00, 5191.21it/s]

2020-03-09 23:18:04,819 [b'4', b'0']


In [8]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings

word_embeddings = [WordEmbeddings('glove'), 
                   #FlairEmbeddings('news-forward'), 
                   #FlairEmbeddings('news-backward')
                  ]

In [9]:
from flair.embeddings import DocumentRNNEmbeddings

document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)

In [10]:
from flair.models import TextClassifier

classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

In [11]:
from flair.trainers import ModelTrainer

trainer = ModelTrainer(classifier, corpus)

In [12]:
trainer.train('model-saves',
              learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=8,
              max_epochs=200)

2020-03-09 23:18:05,877 ----------------------------------------------------------------------------------------------------
2020-03-09 23:18:05,879 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
    )
    (word_reprojection_map): Linear(in_features=100, out_features=256, bias=True)
    (rnn): GRU(256, 512, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=512, out_features=2, bias=True)
  (loss_function): CrossEntropyLoss()
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2020-03-09 23:18:05,880 ----------------------------------------------------------------------------------------------------
2020-03-09 23:18:05,881 Corpus: "Corpus: 160000 train + 20000 dev + 20000 test sentences"
2020-03-09 23:18:05,882 ----------------------------------------------------------------------------------------------------
2020-03-09 2

Process Process-417:
Process Process-419:
Process Process-418:
Process Process-420:


2020-03-10 09:04:44,841 ----------------------------------------------------------------------------------------------------


Process Process-422:
Process Process-421:


2020-03-10 09:04:44,844 Exiting from training early.


Traceback (most recent call last):
Traceback (most recent call last):


2020-03-10 09:04:44,847 Saving model ...


Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._e

2020-03-10 09:04:48,074 Done.
2020-03-10 09:04:48,075 ----------------------------------------------------------------------------------------------------
2020-03-10 09:04:48,076 Testing using best model ...
2020-03-10 09:04:48,078 loading file model-saves/best-model.pt
2020-03-10 09:05:06,877 0.794	0.794	0.794
2020-03-10 09:05:06,880 
MICRO_AVG: acc 0.6584 - f1-score 0.794
MACRO_AVG: acc 0.6583 - f1-score 0.794
0          tp: 7925 - fp: 1917 - fn: 2203 - tn: 7955 - precision: 0.8052 - recall: 0.7825 - accuracy: 0.6579 - f1-score: 0.7937
4          tp: 7955 - fp: 2203 - fn: 1917 - tn: 7925 - precision: 0.7831 - recall: 0.8058 - accuracy: 0.6588 - f1-score: 0.7943
2020-03-10 09:05:06,881 ----------------------------------------------------------------------------------------------------


{'test_score': 0.794,
 'dev_score_history': [0.7317,
  0.7446,
  0.7539,
  0.7652,
  0.7648,
  0.7749,
  0.777,
  0.7739,
  0.7775,
  0.7819,
  0.7798,
  0.7847,
  0.7752,
  0.7813,
  0.786,
  0.7932,
  0.7944,
  0.7904,
  0.7876,
  0.7841,
  0.7969,
  0.7935,
  0.7987,
  0.7938,
  0.8001,
  0.7976,
  0.8007,
  0.8013,
  0.7995,
  0.7993,
  0.7884,
  0.7937,
  0.8026,
  0.7843],
 'train_loss_history': [0.6266119081437588,
  0.5836598127663135,
  0.568054846316576,
  0.5564922459721565,
  0.5492649371296168,
  0.5437442660212517,
  0.5397270953536034,
  0.5343264965295792,
  0.5314284939736128,
  0.5289126058280468,
  0.5255283281028271,
  0.5234941969186068,
  0.5213352509021759,
  0.5188301391690969,
  0.5173505823075771,
  0.5161822039544582,
  0.5137537573516369,
  0.5114896131515503,
  0.510122787898779,
  0.5080120367825032,
  0.5071201262533664,
  0.5058465283244848,
  0.5035509783357381,
  0.5028504619300366,
  0.5015613921701908,
  0.5002922531247139,
  0.4988901760816574,
  0.

In [16]:
from flair.data import Sentence

classifier = TextClassifier.load('model-saves/final-model.pt')

pos_sentence = Sentence(preprocess('I love Python!'))
neg_sentence = Sentence(preprocess('Python is the worst!'))

classifier.predict(pos_sentence)
classifier.predict(neg_sentence)

print(pos_sentence.labels, "\n", neg_sentence.labels)


2020-03-10 09:38:22,686 loading file model-saves/final-model.pt
[4 (0.9295905232429504)] 
 [0 (0.9557440876960754)]
